In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [27]:
train_dir = r'C:\Users\lol51\Downloads\OCT_data\train'
test_dir = r'C:\Users\lol51\Downloads\OCT_data\test'
val_dir = r'C:\Users\lol51\Downloads\OCT_data\val'

In [28]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

Found 10000 images belonging to 4 classes.
Found 100 images belonging to 4 classes.


In [29]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes: NORMAL, CNV, DME, DRUSEN
])

In [30]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
313/313 [==============================] - 198s 629ms/step - loss: 1.0409 - accuracy: 0.5962 - val_loss: 1.1365 - val_accuracy: 0.5200
Epoch 2/10
313/313 [==============================] - 111s 354ms/step - loss: 0.7018 - accuracy: 0.7397 - val_loss: 1.0922 - val_accuracy: 0.5600
Epoch 3/10
313/313 [==============================] - 105s 336ms/step - loss: 0.5258 - accuracy: 0.8088 - val_loss: 0.9878 - val_accuracy: 0.6500
Epoch 4/10
313/313 [==============================] - 106s 339ms/step - loss: 0.3559 - accuracy: 0.8741 - val_loss: 0.9890 - val_accuracy: 0.6700
Epoch 5/10
313/313 [==============================] - 105s 335ms/step - loss: 0.1859 - accuracy: 0.9358 - val_loss: 0.9986 - val_accuracy: 0.6900
Epoch 6/10
313/313 [==============================] - 106s 337ms/step - loss: 0.0678 - accuracy: 0.9809 - val_loss: 1.1012 - val_accuracy: 0.6600
Epoch 7/10
313/313 [==============================] - 109s 348ms/step - loss: 0.0298 - accuracy: 0.9938 - val_loss: 1.2202 -

In [32]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=32, class_mode=None, shuffle=False)
predictions = model.predict(test_generator)

Found 200 images belonging to 1 classes.
7/7 [==============================] - 2s 343ms/step


In [33]:
predicted_labels = tf.argmax(predictions, axis=-1).numpy()

In [35]:
df = pd.DataFrame({'filename': test_generator.filenames, 'label': predicted_labels})
df.to_csv('predictions.csv', index=False)

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [38]:
true_labels = test_generator.classes

In [39]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

C:\Users\lol51\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.385
Precision: 0.25
Recall: 0.09625
F1 Score: 0.13898916967509026
